## インポート

In [182]:
from config import NUMBERS, NUMBERS3, NUMBERS4, NUMBERS3_RESULT, NUMBERS4_RESULT
import sys
import re
import difflib
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

plt.rcParams['font.family'] = 'AppleGothic'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### 調査 その1

In [183]:
X = []
Y = []
Z = []
key_0 = 0
key_1 = 0
key_2 = 0
key_3 = 0
key_4 = 0
key_5 = 0
key_6  = 0 #  赤口
key_7  = 0 #  仏滅
key_8  = 0 #  友引
key_9  = 0 #  先勝
key_10 = 0 #  先負
key_11 = 0 #  大安

"""
先勝・友引・先負・仏滅・大安・赤口
"""
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time]
    _z = NUMBERS[_i][4]
    for _k, _v in enumerate(_x):
        # if len(list(set(_y) & set(_v))) >= 2:
        if len(list(set(_y) & set(_v))) >= 2 and _z == "仏滅":
            if _k == 0:
                key_0 += 1
            elif _k == 1:
                key_1 += 1
            elif _k == 2:
                key_2 += 1
            elif _k == 3:
                key_3 += 1
            elif _k == 4:
                key_4 += 1
            elif _k == 5:
                key_5 += 1
                
            if _z == "赤口":
                key_6 += 1
            elif _z == "仏滅":
                key_7 += 1
            elif _z == "友引":
                key_8 += 1
            elif _z == "先勝":
                key_9 += 1
            elif _z == "先負":
                key_10 += 1
            elif _z == "大安":
                key_11 += 1
            
            
            _format = "{0}回 {1}:{2}({3}) [{4}]".format(_time, _y, _x[_k], _k, _z)
            # print(_time, _y, _v)
            # print(_format)
            pass
_sum     = sum([key_0, key_1, key_2, key_3, key_4, key_5])
_format1 = "前回:{0}個, 前々回:{1}個, 前回曜日:{2}個, 前々回曜日:{3}個, 前回六曜:{4}個, 前々回六曜:{5}個".format(key_0, key_1, key_2, key_3, key_4, key_5)
_format2 = "赤口:{0}個, 仏滅:{1}個, 友引:{2}個 先勝:{3}個, 先負:{4}個, 大安:{5}個".format(key_6, key_7, key_8, key_9, key_10, key_11)

data = {
    "base": [key_0, key_1, key_2, key_3, key_4, key_5],
    "old_week": [key_6, key_7, key_8, key_9, key_10, key_11]
}
_data = pd.DataFrame(data)
print(_format1)
print(_format2)

# sns.countplot(x="old_week", data=_data)


前回:16個, 前々回:20個, 前回曜日:20個, 前々回曜日:28個, 前回六曜:20個, 前々回六曜:22個
赤口:0個, 仏滅:126個, 友引:0個 先勝:0個, 先負:0個, 大安:0個


In [29]:
# 4941
test = ["193", "413", "557", "648", "653", "669"] # 642 -> 464
# 0 -> 0
# 1 -> 2
# 2 -> 0
# 3 -> 2
# 4 -> 2
# 5 -> 3
# 6 -> 3
# 7 -> 1
# 8 -> 1
# 9 -> 2

# 4931
test = ["797", "513", "393", "909", "496", "934"] # 648 -> 480

# 4927 
test = ["393", "531", "851", "402", "934", "604"] # 496 -> 499

# 4919 
test = ["003", "402", "299", "002", "909", "372"] # 604 -> 466

"""
予測したものから、
(y, [x1, x2])
x1: 予測した数字
x2: 重複しなかった数字の合計
"""

'\n予測したものから、\n(y, [x1, x2])\nx1: 予測した数字\nx2: 重複しなかった数字の合計\n'

### 調査2

In [166]:
X = []
Y = []
Z = []
LOCK = {}
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time]
    lock = []
    hook = []
    key_0 = 0
    key_1 = 0
    key_2 = 0
    key_3 = 0
    key_4 = 0
    key_5 = 0
    key_6 = 0
    key_7 = 0
    key_8 = 0
    key_9 = 0
    for _n in _x:
        if _time == _time:
            _h = list(_n)
            for _j in _h:
                if _j == '0':
                    key_0 += 1
                elif _j == '1':
                    key_1 += 1
                elif _j == '2':
                    key_2 += 1
                elif _j == '3':
                    key_3 += 1
                elif _j == '4':
                    key_4 += 1
                elif _j == '5':
                    key_5 += 1
                elif _j == '6':
                    key_6 += 1
                elif _j == '7':
                    key_7 += 1
                elif _j == '8':
                    key_8 += 1
                elif _j == '9':
                    key_9 += 1
            lock.append(list(_n))
    data = {
        "0": str(int(key_0) + 1),
        "1": str(int(key_1) + 1),
        "2": str(int(key_2) + 1),
        "3": str(int(key_3) + 1),
        "4": str(int(key_4) + 1),
        "5": str(int(key_5) + 1),
        "6": str(int(key_6) + 1),
        "7": str(int(key_7) + 1),
        "8": str(int(key_8) + 1),
        "9": str(int(key_9) + 1),
    }
    LOCK.update({ _time: [key_0, key_1, key_2, key_3, key_4, key_5, key_6, key_7, key_8, key_9]})
    Z.append(data)
    X.append(lock)
    
lock = pd.DataFrame(LOCK)

### 1. 前回のナンバーを特徴量とする

In [2]:
X = []
Y = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time][0]
    if len(list(set(_y) & set(_x))) >= 3:
        #print("{0} -> {1}:{2}".format(_time, _y, _x))
        Y.append(_y)
        X.append([_x])

### 2. 前回の六曜のナンバーを特徴量とする

In [33]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time][4]
    if len(list(set(_y) & set(_x))) >= 3:
        # print("{0} -> {1}:{2}".format(_time, _y, _x))
        Y.append(_y)
        X.append([_x])
        TIME.append(_time)

### 3. 前回のナンバーと前回の六曜のナンバーを特徴量とする

In [32]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x1 = NUMBERS3[_i][_time][4]
    _x2 = NUMBERS3[_i][_time][0]
    _z = NUMBERS[_i][4]
    if len(list(set(_y) & set(_x1))) >= 2 or len(list(set(_y) & set(_x2))) >= 2:
        # Y.append(_y)
        # X.append([_x1, _x2])
        # TIME.append(_time)
        pass
    if _z == "先負":
        Y.append(_y)
        X.append([_x1, _x2])
        TIME.append(_time)

### 4. 抽選数を特徴量とする

In [203]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS3_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3_RESULT[_i][_time]
    _z = NUMBERS[_i][4]
    if _z == "仏滅": # 20160625
        Y.append(_y)
        X.append(_x)
        TIME.append(_time)

#### 5. 前回のナンバーと前回曜日ナンバーと前回六曜日のナンバーを特徴量とする

In [140]:
X    = []
Y    = []
TIME = []

"""
先勝・友引・先負・仏滅・大安・赤口
"""

for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x1 = NUMBERS3[_i][_time][0]
    _x2 = NUMBERS3[_i][_time][2]
    _x3 = NUMBERS3[_i][_time][4]
    _z = NUMBERS[_i][4]
    if len(list(set(_y) & set(_x1))) >= 2 or len(list(set(_y) & set(_x2))) >= 2 or _z == "大安":
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)
        pass
    #Y.append(_y)
    #X.append([_x1, _x2, _x3])
    #TIME.append(_time)

#### 6. 前々回のナンバーと前々回曜日のナンバーと前々回六曜のナンバー (仏滅)

In [185]:
X    = []
Y    = []
TIME = []

"""
先勝・友引・先負・仏滅・大安・赤口
"""

for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x1 = NUMBERS3[_i][_time][1]
    _x2 = NUMBERS3[_i][_time][3]
    _x3 = NUMBERS3[_i][_time][5]
    _z = NUMBERS[_i][4]
    if not len(list(set(_y) & set(_x1))) >= 2 and not len(list(set(_y) & set(_x2))) >= 2 and _z == "仏滅":
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)
        pass
    #Y.append(_y)
    #X.append([_x1, _x2, _x3])
    #TIME.append(_time)

#### 前回のナンバーと前回曜日のナンバーと前々回曜日のナンバーを特徴量とする (赤口)

In [33]:
X    = []
Y    = []
TIME = []

"""
先勝・友引・先負・仏滅・大安・赤口
"""

for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x1 = NUMBERS3[_i][_time][0]
    _x2 = NUMBERS3[_i][_time][2]
    _x3 = NUMBERS3[_i][_time][3]
    _z = NUMBERS[_i][4]
    # if len(list(set(_y) & set(_x1))) >= 2 or len(list(set(_y) & set(_x2))) >= 2 or _z == "赤口":
    if len(list(set(_y) & set(_x1))) >= 2 or len(list(set(_y) & set(_x2))) >= 2 or len(list(set(_y) & set(_x3))) and _z == "赤口":
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)
        pass
    # Y.append(_y)
    # X.append([_x1, _x2, _x3])
    # TIME.append(_time)

#### 前々回のナンバーと前回曜日のナンバーと前回六曜のナンバーを特徴量とする (先勝)

In [184]:
X    = []
Y    = []
TIME = []

"""
先勝・友引・先負・仏滅・大安・赤口
"""

for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x1 = NUMBERS3[_i][_time][1]
    _x2 = NUMBERS3[_i][_time][2]
    _x3 = NUMBERS3[_i][_time][4]
    _z = NUMBERS[_i][4]
    # if _z == "先勝":
    if not len(list(set(_y) & set(_x1))) >= 2 and not len(list(set(_y) & set(_x2))) >= 2 and _z == "先勝":
    # if len(list(set(_y) & set(_x1))) >= 2 or len(list(set(_y) & set(_x2))) >= 2 or len(list(set(_y) & set(_x3))) and _z == "赤口":
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)
        pass
    # Y.append(_y)
    # X.append([_x1, _x2, _x3])
    # TIME.append(_time)

#### 予測スタート

In [204]:
y = pd.DataFrame(Y)
x = pd.DataFrame(X)

In [205]:
len(y)

64

In [206]:
model = linear_model.LinearRegression()
model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [207]:
px = x
py = model.predict(px)

In [208]:
NUM = []
for _i in range(len(py)):
    _py = str(int(py[_i]))
    for _k, _v in enumerate(NUMBERS):
        if _v[0] == TIME[_i]:
            if len(list(set(_v[2]) & set(_py))) >= 2:
                _format = "第{0}回: {1}<-{2}   予測:{3}".format(TIME[_i], _v[2], py[_i], LOCK[_v[0]])
                print(_format)
                NUM.append([TIME[_i], _v[2], py[_i], LOCK[_v[0]]])
                pass
            # print(TIME[_i], _v[2], py[_i])
print(len(NUM))

第4893回: 516<-519.5789549162607   予測:[4, 1, 3, 2, 0, 3, 1, 2, 0, 2]
第4826回: 842<-429.2635454189315   予測:[0, 2, 5, 1, 1, 2, 1, 2, 3, 1]
第4789回: 714<-481.54702717344526   予測:[3, 2, 1, 1, 2, 1, 3, 0, 1, 4]
第4748回: 147<-478.7346351361043   予測:[5, 3, 0, 2, 0, 2, 0, 3, 1, 2]
第4633回: 642<-476.55135256441673   予測:[3, 2, 2, 1, 1, 1, 5, 1, 0, 2]
5


## 決定係数

In [209]:
r2_score(y, py)

0.1477094235235793

In [210]:
model.score(x, y)

0.1477094235235793

In [211]:
model.coef_[0]

-1.1499534988577493

## 予測

In [212]:
pkl_file = "data/numbers3_20180625.pkl"

#### 前回のナンバーと前回六曜のナンバー

In [42]:
data = [float("765"), float("642")] # 20180618

##### 前回のナンバー, 前回曜日のナンバー, 前回六曜ナンバー

In [145]:
data = [float("182"), float("642"), float("487")] # 20180620

 #### 前々回のナンバー、前々回曜日のナンバー、前々回六曜のナンバー (仏滅)

In [195]:
data = [float("018"), float("119"), float("182")] # 20180625

#### 前回のナンバー、前回曜日のナンバー、前々回曜のナンバー (赤口)

In [198]:
data = [float("912"), float("487"), float("653")] # 20180621

#### 前々回のナンバー、前回曜日のナンバー、前回六曜のナンバー (先勝)

In [177]:
data = [float("018"), float("119"), float("182")] # 20180622

##### 抽選数

In [213]:
from statistics import mean, median,variance,stdev

X0 = []
X1 = []
X2 = []
X3 = []
for _i in range(len(NUMBERS3_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x0 = NUMBERS3_RESULT[_i][_time][0]
    _x1 = NUMBERS3_RESULT[_i][_time][1]
    _x2 = NUMBERS3_RESULT[_i][_time][2]
    _x3 = NUMBERS3_RESULT[_i][_time][3]
    X0.append(int(_x0))
    X1.append(int(_x1))
    X2.append(int(_x2))
    X3.append(int(_x3))
    

x0 = int(mean(X0))
x1 = int(mean(X1))
x2 = int(mean(X2))
x3 = int(mean(X3))
print(x0, x1, x2, x3)

# 20180618
data = [float("0"), float("0"), float("0"), float("0")]
# data = [float("50"), float("50"), float("50"), float("50")]
# data = [float(str(x0)), float(str(x1)), float(str(x2)), float(str(x3))]

80 448 186 820


In [214]:
joblib.dump(model, pkl_file)

['data/numbers3_20180625.pkl']

In [215]:
clf = joblib.load(pkl_file)

In [216]:
processor = clf.predict([data])

In [217]:
processor

array([831.85439822])